In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()

print("Azure ML Workspace")
print(f'Name: {ws.name}')

Cargamos el modelo

In [ ]:
from azureml.core.model import Model
model = Model(ws, 'concrete-keras')
print(model.name, model.id, model.version, sep='\t')

Creamos el Script de Accesso

In [ ]:
%%writefile score.py
import json
import numpy as np
import os
from keras.models import model_from_json

from azureml.core.model import Model

def init():
    global model
    
    model_root = Model.get_model_path('concrete-keras')
    # load json and create model
    json_file = open(os.path.join(model_root, 'model.json'), 'r')
    model_json = json_file.read()
    json_file.close()
    model = model_from_json(model_json)
    # load weights into new model
    model.load_weights(os.path.join(model_root, "model.h5"))   
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = np.argmax(model.predict(data), axis=1)
    return y_hat.tolist()

Creamos archivo con las Dependencias

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies

cd = CondaDependencies.create()
cd.add_tensorflow_conda_package()
cd.add_conda_package('keras<=2.3.1')
cd.add_pip_package("azureml-defaults")
cd.save_to_file(base_directory='./', conda_file_path='myenv.yml')

print(cd.serialize_to_string())

Publicamos servicio en QA

In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core.environment import Environment


myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                               auth_enabled=True, # this flag generates API keys to secure access
                                               memory_gb=1,
                                               tags={'name': 'concrete', 'framework': 'Keras'},
                                               description='Keras on Concrete')

service = Model.deploy(workspace=ws, 
                           name='keras-concrete-svc', 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)

service.wait_for_deployment(True)
print(service.state)

Funcion de pruebas

In [ ]:
key1, Key2 = service.get_keys()
print(service.scoring_uri)

import requests

def test_endpoint(test_img, y):
    input_data = "{\"data\": [" + str(np.array(test_img).tolist()) + "]}"

    headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + key1}

    resp = requests.post(service.scoring_uri, input_data, headers=headers)

    print("POST to url", service.scoring_uri)
    print("label:", y)
    print("prediction:", resp.text)

Test al servicio web

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

negative_test_img = tf.keras.preprocessing.image.load_img(
    'sample/negative/19751.jpg', grayscale=False, color_mode="rgb", target_size=None, interpolation="nearest"
)

positive_test_img = tf.keras.preprocessing.image.load_img(
    'sample/positive/19754.jpg', grayscale=False, color_mode="rgb", target_size=None, interpolation="nearest"
)

test_endpoint(negative_test_img, 0)
test_endpoint(positive_test_img, 1)

In [ ]:
service.delete()